# Data Preprocessing




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os, os.path, shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pathlib
import os
import datetime
from PIL import Image
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

In [ ]:
# Cell to create training split

# import os, os.path, shutil

# folder_path = "/content/drive/MyDrive/MAIS/MAIS202Data/food-101-data/images"
# train_path = "/content/drive/MyDrive/MAIS/MAIS202Data/food-101-data/test-full"

# folder_path = "/content/food-101/images"
# train_path = "/content/test"

# folders = [f for f in os.listdir(folder_path)]

# for folder in folders:
#     path = os.path.join(folder_path, folder)
#     tpath = os.path.join(train_path,folder)

#     if not os.path.exists(tpath):
#         os.makedirs(tpath)

#     images = os.listdir(path)
#     count = 0
#     for image in images:
#       if count <= 100:
#         old_image_path = os.path.join(path,image)
#         new_image_path = os.path.join(tpath,image)
#         shutil.move(old_image_path, new_image_path)
#       count += 1

In [ ]:
# %cp -a "/content/drive/MyDrive/MAIS/MAIS202Data/food-101-data/image-subset" "/content"
# data_dir = pathlib.Path("/content/image-subset")
# %cp -a "/content/drive/MyDrive/MAIS/MAIS202Data/food-101-data/test" "/content"
# test_dir = pathlib.Path("/content/test")

In [ ]:
!wget https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!tar -xzf food-101.tar.gz
data_dir = pathlib.Path("/content/food-101/images")
val_dir = pathlib.Path("/content/validation")
test_dir = pathlib.Path("/content/test")

exception = [42, 18, 84, 3, 17, 47, 80, 36, 26, 89, 96, 39, 5, 49, 57, 59, 99, 8, 10, 0, 15, 82, 56, 67, 37, 93, 22, 50, 4, 87, 77]

folder_path = data_dir
train_path = test_dir

folders = sorted([f for f in os.listdir(folder_path)])

for index, folder in enumerate(folders):
  path = os.path.join(folder_path, folder)
  tpath = os.path.join(train_path,folder)

  if index in exception:
    shutil.rmtree(path)
    continue

  if not os.path.exists(tpath):
      os.makedirs(tpath)

  images = os.listdir(path)
  count = 0
  for image in images:
    if count <= 100:
      old_image_path = os.path.join(path,image)
      new_image_path = os.path.join(tpath,image)
      shutil.move(old_image_path, new_image_path)
    count += 1

In [ ]:
folder_path = data_dir
val_path = val_dir

folders = [f for f in os.listdir(folder_path)]

for folder in folders:
    path = os.path.join(folder_path, folder)
    tpath = os.path.join(val_path,folder)

    if not os.path.exists(tpath):
        os.makedirs(tpath)

    images = os.listdir(path)
    count = 0
    for image in images:
      if count <= 100:
        old_image_path = os.path.join(path,image)
        new_image_path = os.path.join(tpath,image)
        shutil.move(old_image_path, new_image_path)
      count += 1

In [ ]:
val_dir = test_dir

In [ ]:
batch_size = 64
img_height = 256
img_width = 256

train_img_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  #  validation_split=0.1,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=5,
                                   horizontal_flip=True,
                                   vertical_flip=False,
                                   fill_mode='nearest',
                                   zoom_range=0.2
                                  )

val_img_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_ds = train_img_data_gen.flow_from_directory(directory=data_dir,
                                                    class_mode='sparse',
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    # subset='training',
                                                    shuffle=True
                                                   )


val_ds = val_img_data_gen.flow_from_directory(directory=val_dir,
                                                    class_mode='sparse',
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    # subset='validation',
                                                    shuffle=True
                                                   )

Found 62930 images belonging to 70 classes.
Found 7070 images belonging to 70 classes.


In [ ]:
x_batch, y_batch = next(train_ds)

for i in range(0,11):
    image = x_batch[i]
    plt.imshow(image)
    plt.show()

In [ ]:
num_classes = 70

modelBase = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3), pooling='avg')

modelp = modelBase.output
# modelp = Flatten()(modelp)

# modelp = Dense(2048, kernel_initializer='he_uniform', activation='relu')(modelp)
# modelp = Dense(2048, kernel_initializer='he_uniform', activation='relu')(modelp)
# modelp = Dense(2048, kernel_initializer='he_uniform', activation='relu')(modelp)
modelp = Dense(2048, kernel_initializer='he_uniform', activation='relu')(modelp)
output = Dense(num_classes, activation='softmax')(modelp)

model = Model(inputs=modelBase.input, outputs=output)

for layer in modelBase.layers:
    layer.trainable = False

87916544/87910968 [==============================] - 2s 0us/step


In [ ]:
modelBase = tf.keras.models.load_model('/content/drive/MyDrive/MAIS/MAIS202Data/inception1')
output = Dense(num_classes, activation='relu')(modelBase.layers[-2].output)
model = Model(inputs=modelBase.input, outputs=output)

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/MAIS/MAIS202Data/inception-new2')

In [ ]:
# for i, layer in enumerate(model.layers):
#    print(i, layer.name)

for layer in model.layers[:777]:
   layer.trainable = False
for layer in model.layers[777:]:
   layer.trainable = True

# for layer in model.layers[:781]:
#    layer.trainable = False
# for layer in model.layers[781:]:
#    layer.trainable = True

# for layer in model.layers[:312]:
#    layer.trainable = False
# for layer in model.layers[312:]:
#    layer.trainable = True

In [ ]:
# opt = keras.optimizers.Adam(learning_rate=0.00005)
# opt = keras.optimizers.Adam(learning_rate=0.00001)
# opt = keras.optimizers.Adam(learning_rate=0.000000005)
opt = keras.optimizers.Adam(learning_rate=0.000001)
# opt = keras.optimizers.Adam()
# opt = keras.optimizers.SGD(learning_rate=0.000005, momentum=0.9, nesterov=True)
# opt = keras.optimizers.SGD(momentum=0.9, nesterov=True)

model.compile(
  optimizer=opt,
  loss=tf.losses.SparseCategoricalCrossentropy(),
  metrics=['sparse_categorical_accuracy'])

In [ ]:
print("Start training:",datetime.datetime.now())

modelPath = "/content/drive/MyDrive/MAIS/MAIS202Data/inception-new3"

earlyStopping = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='min')
mcp_save = ModelCheckpoint(modelPath, save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min')

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    verbose=1,
    initial_epoch=0,
    max_queue_size=100,
    workers=300,
    use_multiprocessing=False,
    callbacks=[earlyStopping, mcp_save, reduce_lr_loss]
)

Start training: 2021-04-08 23:35:34.146943
Epoch 1/50
984/984 [==============================] - 715s 646ms/step - loss: 0.2452 - sparse_categorical_accuracy: 0.9295 - val_loss: 0.2169 - val_sparse_categorical_accuracy: 0.9376
INFO:tensorflow:Assets written to: /content/drive/MyDrive/MAIS/MAIS202Data/inception-new3/assets
Epoch 2/50
984/984 [==============================] - 660s 616ms/step - loss: 0.2414 - sparse_categorical_accuracy: 0.9310 - val_loss: 0.2160 - val_sparse_categorical_accuracy: 0.9376
INFO:tensorflow:Assets written to: /content/drive/MyDrive/MAIS/MAIS202Data/inception-new3/assets
Epoch 3/50
984/984 [==============================] - 666s 621ms/step - loss: 0.2394 - sparse_categorical_accuracy: 0.9298 - val_loss: 0.2160 - val_sparse_categorical_accuracy: 0.9379
Epoch 4/50
984/984 [==============================] - 660s 616ms/step - loss: 0.2420 - sparse_categorical_accuracy: 0.9296 - val_loss: 0.2159 - val_sparse_categorical_accuracy: 0.9376
INFO:tensorflow:Assets writ

In [ ]:
# Testing

test_ds = train_img_data_gen.flow_from_directory(directory=test_dir,
                                                    class_mode='categorical',
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    shuffle=True
                                                   )

In [ ]:
print("Evaluate on test data")
results = model.evaluate(test_ds, verbose=1)
print("test loss, test acc:", results)

# Model 6 is current best.
# Model 3: 0.4545. Model 4: 0.4450. Model 5: 0.3918. Model 6: 0.3842 epoch 129

# Full 3: val_loss: 1.5400 - val_accuracy: 0.6430
# Inception new2: 0.2160 0.9379

# Ungenerated full 1: val_loss: 1.5292 - val_accuracy: 0.6542

# Test model 3: loss: 0.4627 - accuracy: 0.8564
# Test model 5: loss: 0.4583 - accuracy: 0.8647

# print("Generate predictions for 3 samples")
# predictions = model.predict(test_ds[:3])
# print("predictions shape:", predictions.shape)

Evaluate on test data
76/76 [==============================] - 22s 288ms/step - loss: 0.4583 - accuracy: 0.8647
test loss, test acc: [0.4582836329936981, 0.8646864891052246]


In [ ]:
!pip install coremltools
import coremltools

In [ ]:
model.save('image_classifier.h5')

image_classifier.short_description = 'Classification of 70 different foods'
image_classifier.input_description['image'] = 'Takes as input an image of a food'
image_classifier.output_description['output'] = 'Prediction of food type'
output_labels = val_ds.class_indices.keys()

your_model = coremltools.converters.keras.convert('image_classifier.h5', input_names=['image'], output_names=['output'], 
                                                   class_labels=output_labels, image_input_names='image', image_scale=2/255.0,
                                                  red_bias=-1, green_bias=-1, blue_bias=-1)

your_model.save('your_model_name.mlmodel')